# Inference demo
In this notebook, you can extract slide embeddings with out pretrained model TITAN.

In [ ]:
import h5py
import torch

from transformers import AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# load model
model = AutoModel.from_pretrained('MahmoodLab/TITAN', trust_remote_code=True)
model = model.to(device)

In [ ]:
# load data
from huggingface_hub import hf_hub_download
demo_h5_path = hf_hub_download(
    "MahmoodLab/TITAN", 
    filename="TCGA_demo_features/TCGA-PC-A5DK-01Z-00-DX1.C2D3BC09-411F-46CF-811B-FDBA7C2A295B.h5",
)
file = h5py.File(demo_h5_path, 'r')
features = torch.from_numpy(file['features'][:])
coords = torch.from_numpy(file['coords'][:])
patch_size_lv0 = file['coords'].attrs['patch_size_level0']

In [ ]:
# extract slide embedding
with torch.autocast(device_type='cuda', dtype=torch.bfloat16), torch.inference_mode():
    features = features.to(device)
    coords = coords.to(device)
    slide_embedding = model.encode_slide_from_patch_features(features, coords, patch_size_lv0)